In [ ]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
def load_roster(filename):
    base = json.load(open(filename,'rb'))

    
    for i in range(len(base['players'])):
        player = base['players'][i]
        if 'name' in player and player['name'] !='':
            sname = base['players'][i]['name'].split()
            base['players'][i]['firstName'] = sname[0]
            base['players'][i]['lastName'] = ' '.join(sname[1:])
            del base['players'][i]['name']
    players = base['players']
    #base['version'] = 32

    return players,base

In [ ]:
players,base = load_roster('nflrosters_2020_Madden_v1.json')
len(players)

In [ ]:
current_ratings =  [_['ratings'][0] for _ in players]
current_ratings = pd.DataFrame(current_ratings)
if 'ovr' in current_ratings.columns:
    current_ratings = current_ratings.drop(['ovr'],1)


In [ ]:
current_ratings.columns

In [ ]:
old_ratings = pd.read_csv('FBGM_League_1_all_seasons_Average_Stats(1).csv')
current_ratings.columns,old_ratings.columns[-20:]

In [ ]:
old_ratings.columns = [_.lower() for _ in old_ratings.columns]
old_ratings['stre'] = old_ratings.str
old_ratings['endu'] = old_ratings.end

In [ ]:
import scipy.stats
stats_correct = {}


for key in current_ratings.columns:
    if key in ['pots','ovrs','pos','season','pot','ovr']:
        continue
    try:
        #print(key)
        orate,crate = old_ratings[key],current_ratings[key]
        plt.figure()
        plt.hist(crate,20,label='base',alpha=0.5,density=True)
        plt.hist(orate,20,label='simulated',alpha=0.5,density=True)
        stats_correct[key] = {'sim': (scipy.stats.tmean(orate),scipy.stats.tstd(orate)), 'base': (scipy.stats.tmean(crate),scipy.stats.tstd(crate))}

        plt.legend()
        plt.title(key)
        
    except KeyboardInterrupt:
        raise
    except:
        print('\t',key)

In [ ]:
if False:
    std_multi = 1.2
    mean_multi = 0.9
    for key in stats_correct:
        stats_correct[key]['base'] = (stats_correct[key]['base'][0]*mean_multi,stats_correct[key]['base'][1])
        stats_correct[key]['sim'] = (stats_correct[key]['sim'][0],stats_correct[key]['sim'][1]*std_multi)

stats_correct

In [ ]:
for player in base['players']:
    for key in player['ratings'][0]:
        if key in stats_correct:
            cv = player['ratings'][0][key]
            z = (cv-stats_correct[key]['base'][0])/stats_correct[key]['base'][1]
            player['ratings'][0][key] = int(round(np.clip(z*stats_correct[key]['sim'][1] + stats_correct[key]['sim'][0],0,100)))

In [ ]:
len(current_ratings.columns)

In [ ]:
players_discard = [_ for _ in base['players']]
current_ratings2 =  [_['ratings'][0] for _ in players]
current_ratings2 = pd.DataFrame(current_ratings2)
if 'ovr' in current_ratings2.columns:
    current_ratings2 = current_ratings2.drop(['ovr'],1)
plt.figure(figsize=(24,24))
for idx,key in enumerate(current_ratings.columns):
    if key in ['pots','ovrs','pos','season','pot','ovr']:
        continue
    orate,crate = old_ratings[key],current_ratings[key]
    w,h = idx//10,idx%10
    plt.subplot(10,8,(1+8*h+w))
    plt.hist(orate,20,label='simulated',alpha=0.5,density=True)
    plt.hist(crate,20,label='base',alpha=0.5,density=True)
    
    plt.legend()
    plt.title(key)
for idx,key in enumerate(current_ratings.columns):
    if key in ['pots','ovrs','pos','season','pot','ovr']:
        continue
    orate,crate = old_ratings[key],current_ratings2[key]
    w,h = idx//10,idx%10
    plt.subplot(10,8,(1+8*h+w+10))
    plt.hist(orate,20,label='simulated',alpha=0.5,density=True)
    plt.hist(crate,20,label='corrected',alpha=0.5,density=True)

    plt.legend()
    plt.title(key)
plt.tight_layout()
plt.savefig('plot.png',edgecolor='w',facecolor='w')

In [ ]:
with open('normed_new.json','wt') as fp:
    json.dump(base,fp, indent=4, sort_keys=True)

In [ ]:
[_ for _ in base['players'] if _['firstName'] == 'Zion']

In [ ]:
[_ for _ in base['players'] if _['firstName'] == 'Zion']